In [2]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [3]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [4]:
import pandas as pd

# Q1 related parameters

In [5]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [39]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [7]:
path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\Single_qubit_design_pp_qubits\Single_qubits_6x10_reduced_to_4.5x8'
path=r'D:\NextCloud\Quantum Computing\ChipDesigns\Ivan_Gerhard_designs\Two_qubits_plus_coupler\Hamiltonian_creation'

In [8]:
data=pd.read_csv(path+'\Design_single_qubit_1.txt',sep='\t')# matrix should be done on pF

In [9]:
data

,Charge,ground,Q1_bottom,Q1_top,Resonator
Charge,0.044879,-0.044700,-0.000094,-0.000077,-0.000009
ground,-0.044700,0.306900,-0.122920,-0.116300,-0.022986
Q1_bottom,-0.000094,-0.122920,0.157590,-0.033236,-0.001340
Q1_top,-0.000077,-0.116300,-0.033236,0.163650,-0.014042
Resonator,-0.000009,-0.022986,-0.001340,-0.014042,0.038377


In [10]:
q_name='Q1_'
Q1_cap_ground_top=data[q_name+'top'][3]+data[q_name+'top'][2]
Q1_cap_ground_bottom=data[q_name+'bottom'][2]+data[q_name+'bottom'][3]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [11]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.033236, 0.124354, 0.13041399999999997)

In [12]:
Q1_cap

0.09689196368460716

In [40]:
Ec(Q1_cap)

199.9

In [14]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(Q2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [31]:
a1    = 0.210
a2    = 0.210 # Junction width in um
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)

Critical current density is:  0.5 uA/um^2
Critical current:  22.049999999999997  nA
Ej:  10.951692140967653  GHz
junction area  in um:  0.04409999999999999


In [32]:
f(Eji(Ic),Ec(Q1_cap)/1e3)

3.985215761538089

In [17]:
Eji(Ic)/(Ec(Q1_cap)/1e3)

60.123048700017996

### Resonator coupling

In [18]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc-Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [19]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [20]:
Q1_cap_res_top=-data[q_name+'top'][-1]
Q1_cap_res_bottom=-data[q_name+'bottom'][-1]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [21]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

14.0
1.3
5.8


In [22]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [118]:
Zr=52.6
omega_r=5
omega_q=4
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

53.7

### Dispersive shift

In [119]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [120]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.4804147270605885

### Readout resonator properties

In [121]:
Q=10000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,4)# first value in ns

(318.3, 0.5)

In [122]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

110.38283802481915

### Charge line

In [76]:
Q1_cap_charge_line_top=-data[q_name+'top'][0]
Q1_cap_charge_line_bottom=-data[q_name+'bottom'][0]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

76.6
93.6
10.5


In [60]:
Cc_charge*1e6

10.545244379730185

In [62]:
omega_q=4

In [63]:
Z0=50
Tcharge=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [64]:
Tcharge*1e6#in us

25026.113067607275

### Total relaxation= resonator+charge line

In [65]:
Tp*Tcharge/(Tp+Tcharge)*1e6

119.71086492265829

In [87]:
# f1=4.5
# f2=4.7
fmin=3.9
fmax=4.7

In [88]:
Ejf(fmin,Ec(Q1_cap)/1e3),Ejf(fmax,Ec(Q1_cap)/1e3)

(10.510993002751375, 15.01314407828915)

In [89]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2

(25.694989203157775, 4.532287336869777)

In [91]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2/j/250,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2/j/100

(205.55991362526217, 90.64574673739553)

In [92]:
np.sqrt(250*205),np.sqrt(90*100)

(226.38462845343543, 94.86832980505137)

In [55]:
# f1=4.5
# f2=4.7
fmin=4
fmax=4

In [56]:
Ejf(fmin,Ec(Q1_cap)/1e3),Ejf(fmax,Ec(Q1_cap)/1e3)

(11.029990001250628, 11.029990001250628)

In [57]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2

(22.207643932738137, 0.0)

In [60]:
(I(fmin,Ec(Q1_cap)/1e3)+I(fmax,Ec(Q1_cap)/1e3))/2/j/210,(I(fmax,Ec(Q1_cap)/1e3)-I(fmin,Ec(Q1_cap)/1e3))/2/j/100

(211.50137078798227, 0.0)

In [105]:
np.sqrt(210*211.5)

210.74866547620178

In [149]:
# flux line relaxation

In [162]:
Z0=50
omega_q=4.2*2*np.pi*1e9

# Ls=0.14e-15
# Cc=0.5e-15

Ls=40e-12
Cc=10e-15

omega_s=1/np.sqrt(Ls*Cc)
Y=1/Z0*(omega_q/omega_s)**4

Tflux=(65e-15)/Y

In [163]:
Tflux*1e6

41.88386424723761